In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost.sklearn import XGBRegressor
from matplotlib import pyplot
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

# data_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'user_data')
# model_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'models')
# output_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'prediction_result')

# data_path = os.path.join(data_dir, 'train_tree.csv')
# test_data_path = os.path.join(data_dir, 'test_tree.csv')
# model_path = os.path.join(model_dir, 'xgb.pkl')
# output_path = os.path.join(output_dir, 'xgb.csv')
data_path = './user_data/train_tree.csv'
test_data_path = './user_data/test_tree.csv'
model_path = './model/xgb.pkl'
output_path = './prediction_result/xgb.csv'

In [2]:
df_train = pd.read_csv(data_path)
df_test = pd.read_csv(test_data_path)
print(df_train.shape)
print(df_test.shape)
y = df_train.pop('price').values
X = df_train.values

(150000, 725)
(50000, 724)


In [8]:
# def my_score(y_predict, y_true):
#     '''
#     自定义评估指标
#     :return:
#     '''
#     score = - mean_absolute_error(np.expm1(y_true), np.nan_to_num(np.expm1(y_predict)))
#     return 'exp_score', score
# score = make_scorer(my_score,greater_is_better=)

def my_score(y_predict, label):
    '''
    自定义评估指标
    :return:
    '''
    label = label.get_label()
    score = mean_absolute_error(np.exp(label), np.exp(y_predict))
    return 'exp_score', score

In [4]:
def log_transfer(func):  #定义一个将数据转为log的闭包函数
    def wrapper(y, yhat):
        result = func(np.exp(y), np.nan_to_num(np.exp(yhat)))
        return result
    return wrapper

## 贝叶斯调参

In [5]:
def xgb_cv(n_estimators, learning_rate, gamma, min_child_weight, max_depth, colsample_bytree, subsample):
    param = {
        'objective': 'reg:squarederror',
        'random_state': 2020,
#         "early_stopping_rounds": 50,
        "tree_method": "gpu_hist",
        "gpu_id": 1
    }
    param['n_estimators'] = int(n_estimators)
    param['learning_rate'] = float(learning_rate)
    param['gamma'] = float(gamma)
    param['min_child_weight'] = float(min_child_weight)
    param['max_depth'] = int(max_depth)
    param['colsample_bytree'] = float(colsample_bytree)
    val = cross_val_score(XGBRegressor(**param),
                          X, y, scoring=make_scorer(log_transfer(mean_absolute_error),greater_is_better=False), cv=5).mean()
    return val


xgb_bo = BayesianOptimization(
    xgb_cv,
    {'n_estimators': (150, 1000),
     'learning_rate': (0.03, 0.3),
     'gamma': (0, 0.5),
     'min_child_weight': (10, 200),
     'max_depth': (4, 10),
     'colsample_bytree': (0.1, 1),
     'subsample': (0.5, 1),
     }
)
# xgb_bo.maximize()
# print(xgb_bo.res)
# print(xgb_bo.max['params'])

In [6]:
X_test = df_test.values

## 使用最佳参数进行训练

In [9]:
from sklearn.model_selection import KFold

param_dist = {
#         "n_estimators": 486,
        "n_estimators": 100000,
        "objective": "reg:squarederror",
        "learning_rate": 0.03,  # [0.03~0.3]
        "gamma": 0.04782946032469271,  # 损失值大于gamma才能继续分割[0~0.5]
        "min_child_weight": 116,  # 最小叶子结点权重 [20~200]
        "max_depth": 8,  # [4~10]
        "colsample_bytree": 0.7237044081916191,  # [0.3~1]
        "subsample": 0.985,  # [0.3~1]
        "tree_method": "gpu_hist",
        "gpu_id": 1
    }

folds = KFold(n_splits=5, shuffle=True, random_state=2020)
oof_xgb = np.zeros(len(X))
predictions_xgb = np.zeros(len(X_test))
predictions_train_xgb = np.zeros(len(X))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print("fold n°{}".format(fold_ + 1))
    clf = XGBRegressor(**param_dist)
    clf.fit(X[trn_idx],y[trn_idx], eval_set=[(X[val_idx], y[val_idx])], eval_metric=my_score,early_stopping_rounds=50)
    oof_xgb[val_idx] = clf.predict(X[val_idx],ntree_limit=clf.best_iteration)
    predictions_xgb += clf.predict(X_test, ntree_limit=clf.best_iteration) / folds.n_splits
    predictions_train_xgb += clf.predict(X, ntree_limit=clf.best_iteration) / folds.n_splits
#     # 预测验证集
#     oof_xgb[val_idx] = clf.predict(X[val_idx])
#     # 预测测试集
#     predictions_xgb += clf.predict(X_test) / folds.n_splits
#     # 预测完整的测试集(包含验证集)
#     predictions_train_xgb += clf.predict(X) / folds.n_splits

fold n°1
[0]	validation_0-rmse:7.40027	validation_0-exp_score:5905.85547
Multiple eval metrics have been passed: 'validation_0-exp_score' will be used for early stopping.

Will train until validation_0-exp_score hasn't improved in 50 rounds.
[1]	validation_0-rmse:7.17881	validation_0-exp_score:5905.34473
[2]	validation_0-rmse:6.96396	validation_0-exp_score:5904.72510
[3]	validation_0-rmse:6.75556	validation_0-exp_score:5903.97803
[4]	validation_0-rmse:6.55341	validation_0-exp_score:5903.08350
[5]	validation_0-rmse:6.35742	validation_0-exp_score:5902.01709
[6]	validation_0-rmse:6.16725	validation_0-exp_score:5900.75098
[7]	validation_0-rmse:5.98284	validation_0-exp_score:5899.25928
[8]	validation_0-rmse:5.80391	validation_0-exp_score:5897.50732
[9]	validation_0-rmse:5.63041	validation_0-exp_score:5895.46338
[10]	validation_0-rmse:5.46208	validation_0-exp_score:5893.08789
[11]	validation_0-rmse:5.29880	validation_0-exp_score:5890.34082
[12]	validation_0-rmse:5.14047	validation_0-exp_scor

[124]	validation_0-rmse:0.29760	validation_0-exp_score:1272.80786
[125]	validation_0-rmse:0.29445	validation_0-exp_score:1248.31909
[126]	validation_0-rmse:0.29154	validation_0-exp_score:1224.58032
[127]	validation_0-rmse:0.28869	validation_0-exp_score:1201.48682
[128]	validation_0-rmse:0.28596	validation_0-exp_score:1179.22693
[129]	validation_0-rmse:0.28343	validation_0-exp_score:1157.56885
[130]	validation_0-rmse:0.28101	validation_0-exp_score:1136.48499
[131]	validation_0-rmse:0.27876	validation_0-exp_score:1115.92139
[132]	validation_0-rmse:0.27649	validation_0-exp_score:1096.26892
[133]	validation_0-rmse:0.27447	validation_0-exp_score:1077.33496
[134]	validation_0-rmse:0.27252	validation_0-exp_score:1058.97180
[135]	validation_0-rmse:0.27067	validation_0-exp_score:1040.93603
[136]	validation_0-rmse:0.26892	validation_0-exp_score:1023.61102
[137]	validation_0-rmse:0.26551	validation_0-exp_score:1004.66888
[138]	validation_0-rmse:0.26395	validation_0-exp_score:988.40320
[139]	valid

[250]	validation_0-rmse:0.23400	validation_0-exp_score:581.79279
[251]	validation_0-rmse:0.23395	validation_0-exp_score:581.59106
[252]	validation_0-rmse:0.23390	validation_0-exp_score:581.38104
[253]	validation_0-rmse:0.23384	validation_0-exp_score:581.20013
[254]	validation_0-rmse:0.23379	validation_0-exp_score:580.69971
[255]	validation_0-rmse:0.23370	validation_0-exp_score:580.42639
[256]	validation_0-rmse:0.23363	validation_0-exp_score:579.88165
[257]	validation_0-rmse:0.23361	validation_0-exp_score:579.71918
[258]	validation_0-rmse:0.23356	validation_0-exp_score:579.58740
[259]	validation_0-rmse:0.23350	validation_0-exp_score:579.24225
[260]	validation_0-rmse:0.23346	validation_0-exp_score:579.05969
[261]	validation_0-rmse:0.23343	validation_0-exp_score:578.91217
[262]	validation_0-rmse:0.23342	validation_0-exp_score:578.76062
[263]	validation_0-rmse:0.23337	validation_0-exp_score:578.49078
[264]	validation_0-rmse:0.23334	validation_0-exp_score:578.34766
[265]	validation_0-rmse:0

[377]	validation_0-rmse:0.23010	validation_0-exp_score:563.80005
[378]	validation_0-rmse:0.23008	validation_0-exp_score:563.66949
[379]	validation_0-rmse:0.23007	validation_0-exp_score:563.59051
[380]	validation_0-rmse:0.23004	validation_0-exp_score:563.54541
[381]	validation_0-rmse:0.23003	validation_0-exp_score:563.48401
[382]	validation_0-rmse:0.23000	validation_0-exp_score:563.37421
[383]	validation_0-rmse:0.23000	validation_0-exp_score:563.20709
[384]	validation_0-rmse:0.22999	validation_0-exp_score:563.12793
[385]	validation_0-rmse:0.22998	validation_0-exp_score:563.06512
[386]	validation_0-rmse:0.22997	validation_0-exp_score:563.01208
[387]	validation_0-rmse:0.22994	validation_0-exp_score:562.73669
[388]	validation_0-rmse:0.22992	validation_0-exp_score:562.63123
[389]	validation_0-rmse:0.22991	validation_0-exp_score:562.64673
[390]	validation_0-rmse:0.22989	validation_0-exp_score:562.60290
[391]	validation_0-rmse:0.22986	validation_0-exp_score:562.31775
[392]	validation_0-rmse:0

[504]	validation_0-rmse:0.22837	validation_0-exp_score:552.56274
[505]	validation_0-rmse:0.22837	validation_0-exp_score:552.54669
[506]	validation_0-rmse:0.22832	validation_0-exp_score:552.34619
[507]	validation_0-rmse:0.22831	validation_0-exp_score:552.22790
[508]	validation_0-rmse:0.22830	validation_0-exp_score:552.20813
[509]	validation_0-rmse:0.22831	validation_0-exp_score:552.18280
[510]	validation_0-rmse:0.22828	validation_0-exp_score:551.79077
[511]	validation_0-rmse:0.22826	validation_0-exp_score:551.77631
[512]	validation_0-rmse:0.22827	validation_0-exp_score:551.78082
[513]	validation_0-rmse:0.22826	validation_0-exp_score:551.75085
[514]	validation_0-rmse:0.22826	validation_0-exp_score:551.71533
[515]	validation_0-rmse:0.22826	validation_0-exp_score:551.67175
[516]	validation_0-rmse:0.22824	validation_0-exp_score:551.59900
[517]	validation_0-rmse:0.22823	validation_0-exp_score:551.59882
[518]	validation_0-rmse:0.22821	validation_0-exp_score:551.42920
[519]	validation_0-rmse:0

...

[1776]	validation_0-rmse:0.21868	validation_0-exp_score:512.14667
[1777]	validation_0-rmse:0.21868	validation_0-exp_score:512.14313
[1778]	validation_0-rmse:0.21868	validation_0-exp_score:512.16302
[1779]	validation_0-rmse:0.21868	validation_0-exp_score:512.07391
[1780]	validation_0-rmse:0.21869	validation_0-exp_score:512.06158
[1781]	validation_0-rmse:0.21868	validation_0-exp_score:512.07965
[1782]	validation_0-rmse:0.21868	validation_0-exp_score:512.03766
[1783]	validation_0-rmse:0.21867	validation_0-exp_score:512.03156
[1784]	validation_0-rmse:0.21867	validation_0-exp_score:512.00665
[1785]	validation_0-rmse:0.21867	validation_0-exp_score:512.00726
[1786]	validation_0-rmse:0.21867	validation_0-exp_score:512.00745
[1787]	validation_0-rmse:0.21867	validation_0-exp_score:511.95282
[1788]	validation_0-rmse:0.21866	validation_0-exp_score:511.82935
[1789]	validation_0-rmse:0.21866	validation_0-exp_score:511.82935
[1790]	validation_0-rmse:0.21866	validation_0-exp_score:511.79056
[1791]	val

[1901]	validation_0-rmse:0.21858	validation_0-exp_score:510.60529
[1902]	validation_0-rmse:0.21858	validation_0-exp_score:510.60580
[1903]	validation_0-rmse:0.21857	validation_0-exp_score:510.58627
[1904]	validation_0-rmse:0.21857	validation_0-exp_score:510.55780
[1905]	validation_0-rmse:0.21857	validation_0-exp_score:510.52393
[1906]	validation_0-rmse:0.21857	validation_0-exp_score:510.52374
[1907]	validation_0-rmse:0.21857	validation_0-exp_score:510.52301
[1908]	validation_0-rmse:0.21857	validation_0-exp_score:510.52344
[1909]	validation_0-rmse:0.21857	validation_0-exp_score:510.50253
[1910]	validation_0-rmse:0.21857	validation_0-exp_score:510.50293
[1911]	validation_0-rmse:0.21857	validation_0-exp_score:510.50293
[1912]	validation_0-rmse:0.21857	validation_0-exp_score:510.50269
[1913]	validation_0-rmse:0.21857	validation_0-exp_score:510.49695
[1914]	validation_0-rmse:0.21858	validation_0-exp_score:510.48752
[1915]	validation_0-rmse:0.21858	validation_0-exp_score:510.48688
[1916]	val

[2026]	validation_0-rmse:0.21859	validation_0-exp_score:509.98544
[2027]	validation_0-rmse:0.21859	validation_0-exp_score:509.98254
[2028]	validation_0-rmse:0.21859	validation_0-exp_score:509.98367
[2029]	validation_0-rmse:0.21859	validation_0-exp_score:509.98386
[2030]	validation_0-rmse:0.21859	validation_0-exp_score:509.96088
[2031]	validation_0-rmse:0.21859	validation_0-exp_score:509.96106
[2032]	validation_0-rmse:0.21859	validation_0-exp_score:509.96146
[2033]	validation_0-rmse:0.21859	validation_0-exp_score:509.96155
[2034]	validation_0-rmse:0.21859	validation_0-exp_score:509.96170
[2035]	validation_0-rmse:0.21859	validation_0-exp_score:509.96170
[2036]	validation_0-rmse:0.21859	validation_0-exp_score:509.96170
[2037]	validation_0-rmse:0.21859	validation_0-exp_score:509.96237
[2038]	validation_0-rmse:0.21859	validation_0-exp_score:509.96286
[2039]	validation_0-rmse:0.21858	validation_0-exp_score:509.93619
[2040]	validation_0-rmse:0.21858	validation_0-exp_score:509.93643
[2041]	val

[2151]	validation_0-rmse:0.21857	validation_0-exp_score:509.45990
[2152]	validation_0-rmse:0.21857	validation_0-exp_score:509.46033
[2153]	validation_0-rmse:0.21857	validation_0-exp_score:509.46069
[2154]	validation_0-rmse:0.21857	validation_0-exp_score:509.46069
[2155]	validation_0-rmse:0.21857	validation_0-exp_score:509.46069
[2156]	validation_0-rmse:0.21857	validation_0-exp_score:509.46069
[2157]	validation_0-rmse:0.21857	validation_0-exp_score:509.46091
[2158]	validation_0-rmse:0.21857	validation_0-exp_score:509.46048
[2159]	validation_0-rmse:0.21857	validation_0-exp_score:509.46323
[2160]	validation_0-rmse:0.21857	validation_0-exp_score:509.46344
[2161]	validation_0-rmse:0.21857	validation_0-exp_score:509.45825
[2162]	validation_0-rmse:0.21857	validation_0-exp_score:509.45825
[2163]	validation_0-rmse:0.21857	validation_0-exp_score:509.45807
[2164]	validation_0-rmse:0.21857	validation_0-exp_score:509.44894
[2165]	validation_0-rmse:0.21857	validation_0-exp_score:509.44916
[2166]	val

[2276]	validation_0-rmse:0.21854	validation_0-exp_score:509.08154
[2277]	validation_0-rmse:0.21854	validation_0-exp_score:509.08154
[2278]	validation_0-rmse:0.21854	validation_0-exp_score:509.08173
[2279]	validation_0-rmse:0.21855	validation_0-exp_score:509.09378
[2280]	validation_0-rmse:0.21854	validation_0-exp_score:509.07059
[2281]	validation_0-rmse:0.21854	validation_0-exp_score:509.07037
[2282]	validation_0-rmse:0.21854	validation_0-exp_score:509.07074
[2283]	validation_0-rmse:0.21854	validation_0-exp_score:509.07117
[2284]	validation_0-rmse:0.21854	validation_0-exp_score:509.07178
[2285]	validation_0-rmse:0.21854	validation_0-exp_score:509.07193
[2286]	validation_0-rmse:0.21854	validation_0-exp_score:509.07196
[2287]	validation_0-rmse:0.21854	validation_0-exp_score:509.07196
[2288]	validation_0-rmse:0.21854	validation_0-exp_score:509.07181
[2289]	validation_0-rmse:0.21855	validation_0-exp_score:509.06421
[2290]	validation_0-rmse:0.21855	validation_0-exp_score:509.03906
[2291]	val

In [ ]:
print("xgb score: {:<8.8f}".format(mean_absolute_error(np.exp(oof_xgb), np.expm1(y))))

# 验证集输出
SaleID = range(150000)
y_true = y
y_predict = oof_xgb
df = pd.DataFrame({'SaleID':SaleID,'y_true':y_true,'y_predict':y_predict})
df.to_csv('./prediction_result/xgb_valid.csv',index=False)

In [ ]:
# 测试集输出
saleID = np.array(range(200000, 250000))
price = np.exp(predictions_xgb)
df = pd.DataFrame({'SaleID':saleID,'price':price})
df.to_csv('./prediction_result/xgb_test.csv',index=False)

In [ ]:
df.describe()

In [ ]:
# clf.importance_type='gain'
im = pd.DataFrame({'feature':df_train.columns,'importance':clf.feature_importances_})
im=im.sort_values('importance',ascending=False)

In [ ]:
im.head(50)

In [ ]:
import seaborn as sns
f, ax = plt.subplots(figsize=(16, 16))
sns.barplot(y='feature',x='importance',data=im.head(50))